# Notebook 17: Histogram with Atomic Operations## Phase 3: Optimization Fundamentals**Learning Objectives:**- Understand atomic operations- Learn histogram computation- Master privatization- Apply concepts in practical scenarios- Measure and analyze performance

## Concept: Histogram with Atomic Operations**Topics Covered:**- atomic operations- histogram computation- privatization**Key Concepts:**This notebook covers atomic operations in the context of Phase 3: Optimization Fundamentals.

## Example 1: Basic Histogram with Atomic Operations

In [ ]:
%%cu#include <stdio.h>#include <cuda_runtime.h>#define CUDA_CHECK(call) \    do { \        cudaError_t err = call; \        if (err != cudaSuccess) { \            fprintf(stderr, "CUDA error: %s\n", cudaGetErrorString(err)); \            exit(EXIT_FAILURE); \        } \    } while(0)#define NUM_BINS 256__global__ void histogramKernel(unsigned char *data, int *histogram, int n) {    __shared__ int smem[NUM_BINS];    int tid = threadIdx.x;    int idx = blockIdx.x * blockDim.x + threadIdx.x;    if (tid < NUM_BINS) smem[tid] = 0;    __syncthreads();    if (idx < n) atomicAdd(&smem[data[idx]], 1);    __syncthreads();    if (tid < NUM_BINS) atomicAdd(&histogram[tid], smem[tid]);}int main() {    printf("=== Histogram ===\n\n");    int n = 1 << 20;    unsigned char *d_data;    int *d_histogram;    CUDA_CHECK(cudaMalloc(&d_data, n));    CUDA_CHECK(cudaMalloc(&d_histogram, NUM_BINS * sizeof(int)));    CUDA_CHECK(cudaMemset(d_histogram, 0, NUM_BINS * sizeof(int)));    int threads = 256;    histogramKernel<<<(n+threads-1)/threads, threads>>>(d_data, d_histogram, n);    printf("Histogram computed\n");    cudaFree(d_data);    cudaFree(d_histogram);    return 0;}

## Practical ExerciseComplete the following exercises to practice the concepts learned.

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", \
                    __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

__global__ void kernel(float *data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}

int main() {
    printf("=== Histogram ===\n\n");

    int n = 1000000;
    size_t size = n * sizeof(float);

    float *h_data = (float*)malloc(size);
    for (int i = 0; i < n; i++) h_data[i] = i;

    float *d_data;
    CUDA_CHECK(cudaMalloc(&d_data, size));
    CUDA_CHECK(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int threads = 256;
    int blocks = (n + threads - 1) / threads;

    cudaEventRecord(start);
    kernel<<<blocks, threads>>>(d_data, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);

    CUDA_CHECK(cudaMemcpy(h_data, d_data, size, cudaMemcpyDeviceToHost));

    printf("Processed %d elements in %.2f ms\n", n, ms);
    printf("Bandwidth: %.2f GB/s\n", (size * 2 / 1e9) / (ms / 1000.0));

    free(h_data);
    cudaFree(d_data);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

## Key Takeaways

1. Histogram counts occurrences in bins
2. Atomic operations for thread-safe updates
3. Privatization reduces atomic contention
4. Multiple optimization strategies

## Next StepsContinue to: **18_next_topic.ipynb**

## Notes*Use this space to write your own notes and observations:*------